In [1]:
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
import sys
import os

# Add the directory where custom_env.py is located
sys.path.append('/content/')

In [3]:
import os
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.logger import configure
from custom_env import EducationHubEnv  # Your custom environment

# Custom callback to log rewards
class EducationLoggerCallback(BaseCallback):
    def __init__(self, log_dir, verbose=1):
        super(EducationLoggerCallback, self).__init__(verbose)
        self.episode_rewards = []
        self.entropy_values = []
        self.log_dir = log_dir
        os.makedirs(log_dir, exist_ok=True)

    def _on_step(self) -> bool:
        if "infos" in self.locals and len(self.locals["infos"]) > 0:
            episode_reward = self.locals["infos"][0].get("episode", {}).get("r")
            if episode_reward is not None:
                self.episode_rewards.append(episode_reward)
                print(f"Episode Reward Logged: {episode_reward}")

        entropy = float(self.model.logger.name_to_value.get("entropy_loss", 0))
        self.entropy_values.append(entropy)
        print(f"Entropy Logged: {entropy}")

        return True

    def on_training_end(self):
        np.save(os.path.join(self.log_dir, "reward_history.npy"), np.array(self.episode_rewards, dtype=np.float32))
        np.save(os.path.join(self.log_dir, "entropy_history.npy"), np.array(self.entropy_values, dtype=np.float32))

def train_ppo():
    env = EducationHubEnv()
    models_dir = "models/ppo/"
    log_dir = "logs/ppo/"
    best_model_dir = os.path.join(models_dir, "best/")

    os.makedirs(models_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(best_model_dir, exist_ok=True)

    model = PPO(
        "MlpPolicy",  # Use MLP since observations are not dict-based
        env,
        verbose=1,
        learning_rate=3e-4,
        gamma=0.99,
        batch_size=64,
        n_steps=2048,
        n_epochs=10,
        ent_coef=0.01,
        tensorboard_log=log_dir
    )

    logger = configure(log_dir, ["stdout", "tensorboard"])
    model.set_logger(logger)

    reward_logger = EducationLoggerCallback(log_dir)
    eval_callback = EvalCallback(
        env,
        best_model_save_path=best_model_dir,
        eval_freq=5000,
        deterministic=True,
        render=False
    )

    model.learn(total_timesteps=100000, callback=[reward_logger, eval_callback])
    model.save(os.path.join(models_dir, "ppo_final_model"))
    print("Training complete. PPO model saved.")

if __name__ == "__main__":
    train_ppo()


/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


Streaming output truncated to the last 5000 lines.
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: -1.6
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: -9.6
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: -11.6
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: -9.6
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: -9.6
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: -10.6
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: -4.6
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: 1.4
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: -1.6
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: -4.6
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: 1.4
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: 1.4
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Entropy Logged: 0.0
Episode Reward Logged: -